In [3]:
import sys
sys.path.append("../")
import pandas as pd

from utils.prompts import render
from utils.llm_client import LLMClient
from utils.logging_utils import log_llm_call
from utils.router import pick_model,should_use_reasoning_model
from IPython.display import display, Markdown

In [5]:
with open('../data/Sample Messages.txt') as f:
    sample_messages = f.read().strip().split('\n\n')
    for message in sample_messages:
        print(message)
        print()  # blank line between messages

BREAKING: Water levels in Kelani River (Colombo) have reached 9.5 meters. Critical flood warning issued.

SOS: 5 people trapped on a roof in Ja-Ela (Gampaha). Water rising fast. Need boat immediately.

Update: Kandy road cleared near Peradeniya. Traffic moving slowly. No victims reported.

Does anyone have extra dry rations for the camp in Gampaha?

News just in: Kelani river water level is at 7ft.

My uncle is stuck in the tree (immediate danger).

Just saw on news that Gampaha town is flooded. Hope everyone is safe.

We are trapped in the attic. 3 kids. Water is entering.

Donation drive happening at the town hall tomorrow.

Report: Heavy rains expected to continue for the next 24 hours.

URGENT: Landslide in Badulla. 2 houses buried. Need rescue team.

Can someone recharge my phone? 077-1234567.

The Navy has deployed 5 boats to the Kelaniya area.

Help! My grandmother is bedridden and the water is knee deep. Location: Kolonnawa.

Is the highway open? I need to get to the airport.

